# Glossary

## Usage

```
pv command --parameterName value
```

Note:

- **( )** (parens) required elements.
- **\[ \]** (brackets) optional elements.
- **|** (pipe) mutually exclusive elements.
- **...** (ellipsis) one or more elements.

## Commands

**Glossary**

```
getGlossary [--glossaryGuid=<glossaryGuid>] [--limit=<limit> --offset=<offset> --sort=<sort>]
getGlossaryDetailed (--glossaryGuid=<glossaryGuid>)
deleteGlossary (--glossaryGuid=<glossaryGuid>)
```

**Category**

```
createGlossaryCategory (--categoryName=<name>) [--glossaryGuid=<glossaryGuid>]
createGlossaryCategories (--categoryName=<name>...) [--glossaryGuid=<glossaryGuid>]
getGlossaryCategory (--categoryGuid=<categoryGuid>)
deleteGlossaryCategory (--categoryGuid=<categoryGuid>)
getGlossaryCategories (--glossaryGuid=<glossaryGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
getGlossaryCategoriesHeaders (--glossaryGuid=<glossaryGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
getGlossaryCategoryRelated (--categoryGuid=<categoryGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
getGlossaryCategoryTerms (--categoryGuid=<categoryGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
```

**Terms**

```
createGlossaryTerm (--termName=<termName>) [--glossaryGuid=<glossaryGuid> --status=<status> --longDescription=<longDescription> --abbreviation=<abbreviation> --synonym=<synonym>... --related=<related>... --resourceName=<resourceName>... --resourceUrl=<resourceUrl>... --expertId=<expertId>... --stewardId=<stewardId>...]
createGlossaryTerms (--termName=<termName>...) [--glossaryGuid=<glossaryGuid> --status=<status>... --longDescription=<longDescription>...]
getGlossaryTerm (--termGuid=<termGuid>)
updateGlossaryTerm (--termGuid=<termGuid>) [--termName=<termName> --glossaryGuid=<glossaryGuid> --status=<status> --longDescription=<longDescription> --abbreviation=<abbreviation> --synonym=<synonym>... --related=<related>... --resourceName=<resourceName>... --resourceUrl=<resourceUrl>... --expertId=<expertId>... --stewardId=<stewardId>...]
deleteGlossaryTerm (--termGuid=<termGuid>)
purgeGlossaryTerms [--glossaryGuid=<glossaryGuid>]
getGlossaryTerms (--glossaryGuid=<glossaryGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
getGlossaryTermsHeaders (--glossaryGuid=<glossaryGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
getGlossaryTermsRelated (--termGuid=<termGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
```

**Assigned Entities**

```
assignEntities (--termGuid=<termGuid> --guid=<guid>...)
getAssignedEntities (--termGuid=<termGuid>) [--limit=<limit> --offset=<offset> --sort=<sort>]
deleteAssignedEntities (--termGuid=<termGuid> --guid=<guid>...)
```

**Template**

```
getGlossaryTemplate
```

### **Pre-requisite**

You will need to install the purviewcli package.

```
pip install purviewcli
```

### **1\. Set the Environment Variable**

Before purviewcli can be used, the environment variable `PURVIEW_NAME` needs to be set.

In [ ]:
 $env:PURVIEW_NAME = "purview-sandbox"
 Write-Host "Environment Variable PURVIEW_NAME set to $env:PURVIEW_NAME."

### **2\. Get Glossary**

Retrieve all glossaries registered with Azure Purview.

In [ ]:
$glossary = pv getGlossary | ConvertFrom-Json
$glossary

### **3\. Create Glossary Term**

Create a glossary term.

In [ ]:
$term = pv createGlossaryTerm --termName 'My First Term' --glossaryGuid ($glossary[0].guid) --status 'Draft' --longDescription 'This is a term definition.' | ConvertFrom-Json
$term

### **4\. Get Glossary Term**

Get specific glossary term.

In [ ]:
pv getGlossaryTerm --termGuid ($term.guid)

### **5\. Update Glossary Term**

Update the given glossary term.

In [ ]:
pv updateGlossaryTerm --termGuid ($term.guid) --status 'Approved'

### **6\. Delete Glossary Term**

Delete a glossary term.

In [ ]:
pv deleteGlossaryTerm --termGuid ($term.guid)

### **7\. Create Glossary Terms (Bulk)**

Create glossary terms in bulk.

In [ ]:
pv createGlossaryTerms `
--termName 'Term1' --status 'Draft' --longDescription 'Long description for Term1.' `
--termName 'Term2' --status 'Approved' --longDescription 'Long description for Term2.' `
--termName 'Term3' --status 'Expired' --longDescription 'Long description for Term3.' `
--termName 'Term4' --status 'Alert' --longDescription 'Long description for Term4.'

### **8\. Purge Glossary Terms (Bulk Deletion)**

**<u><mark>WARNING:</mark></u>** This command will delete ALL terms from the glossary and remove ALL term assignments.

In [ ]:
# IMPORTANT: Uncommenting and running the command below will purge the glossary of ALL terms.
# pv purgeGlossaryTerms

### **9\. Create Glossary Terms (Bulk via CSV)**

Create glossary terms in bulk via a CSV file.

In [ ]:
# Initialise Variables
$filepath = '/Users/taygan/Downloads/AdventureWorks/DataDictionary.csv'
$counter = 0
$batch_size = 100
$command = 'pv createGlossaryTerms'
$csv = Import-CSV $filepath

# Loop through rows in csv
$csv | Foreach-Object{
   $counter += 1
   $command += " --termName ""$($_.table): $($_.column)"" --status ""Approved"" --longDescription ""$($_.description)"""
   if ($counter % $batch_size -eq 0) {
      Write-Host "[INFO] Uploading $($counter) of $($csv.length)"
      $results = Invoke-Expression $command
      $command = 'pv createGlossaryTerms'
   }
}
Write-Host "[INFO] Uploading $($counter) of $($csv.length)"
$results = Invoke-Expression $command
Write-Host "[INFO] Bulk upload complete."